In [1]:
from sklearn import tree
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
# 导入自己写的工具类
from my_tools import *
# 忽略warning
import warnings
warnings.filterwarnings("ignore")

In [2]:
jibing_res = pd.read_excel("./jibing_feature_res_final.xlsx")
jibing = pd.read_excel("./jibing_feature_final.xlsx")

## 在SVM中连续型变量应尽可能归一化
### 11到60列是连续型的变量，要归一化

In [3]:
jibing.iloc[:,11:62].head()

,红细胞计数*10^12/L,血红蛋白,红细胞压积,血小板计数,血小板压积,总蛋白g/L,白蛋白g/L,球蛋白g/L,白球比,ALT丙氨酸氨基转移酶,...,腺苷脱氨酶ADA,果糖胺,肌酸激酶,α-L-盐藻糖苷酶,乳酸,淀粉酶,同型半胱氨酸,铁,总铁结合力,血型
0,4.90,143.0,43.0,233.0,0.205,84.5,51.5,33.0,1.6,24,...,17.0,2.11,67.0,28.0,1.4,77.0,13.2,12.5,59.7,2
1,4.79,145.0,43.7,243.0,0.231,81.0,47.8,33.2,1.4,50,...,15.0,2.01,98.0,27.0,2.6,71.0,11.8,19.9,59.5,2
2,4.23,132.0,39.6,287.0,0.240,65.4,39.8,25.6,1.6,12,...,10.0,1.75,104.0,19.0,2.9,46.0,11.8,18.3,66.9,2
3,4.25,139.0,41.1,164.0,0.140,72.2,44.9,27.3,1.6,18,...,12.0,2.35,91.0,21.0,1.8,101.0,12.9,18.3,48.7,3
4,3.70,119.0,35.2,208.0,0.221,71.8,37.5,34.3,1.1,8,...,12.0,1.66,57.0,21.0,1.8,87.0,10.1,20.8,43.0,1


In [4]:
jibing.index = range(jibing.shape[0])

## 归一化

In [5]:
jibing = guiyihua(jibing)

In [6]:
jibing.head()

,左右,是否外伤,症状持续时间,明显夜间痛,性别,年龄,高血压,高血脂,2型糖尿病,吸烟与否,...,腺苷脱氨酶ADA,果糖胺,肌酸激酶,α-L-盐藻糖苷酶,乳酸,淀粉酶,同型半胱氨酸,铁,总铁结合力,血型
0,1,1,4,1,0,0.083333,0,0,1,0,...,0.488327,0.005899,0.005654,0.195489,0.100775,0.264014,0.081800,0.240275,0.515732,2
1,1,0,4,0,0,0.105856,1,0,0,0,...,0.424368,0.005321,0.008347,0.187970,0.193798,0.238340,0.067485,0.409611,0.512996,2
2,1,0,4,1,0,0.101351,0,0,0,0,...,0.264471,0.003817,0.008868,0.127820,0.217054,0.131365,0.067485,0.372998,0.614227,2
3,1,0,4,1,0,0.128378,1,0,1,0,...,0.328430,0.007287,0.007739,0.142857,0.131783,0.366709,0.078732,0.372998,0.365253,3
4,1,0,4,0,0,0.105856,1,0,0,0,...,0.328430,0.003297,0.004786,0.142857,0.131783,0.306804,0.050102,0.430206,0.287278,1


### 标准化

In [7]:
jibing = biaozhunhua(jibing)

In [8]:
jibing.head()

,左右,是否外伤,症状持续时间,明显夜间痛,性别,年龄,高血压,高血脂,2型糖尿病,吸烟与否,...,腺苷脱氨酶ADA,果糖胺,肌酸激酶,α-L-盐藻糖苷酶,乳酸,淀粉酶,同型半胱氨酸,铁,总铁结合力,血型
0,1,1,4,1,0,-0.487123,0,0,1,0,...,1.623080,0.022580,-0.125125,0.627545,-0.390934,0.438800,0.135186,-0.836831,0.519737,2
1,1,0,4,0,0,0.256311,1,0,0,0,...,1.045433,-0.000756,-0.041179,0.510785,1.196191,0.182967,-0.101026,0.360809,0.498639,2
2,1,0,4,1,0,0.107624,0,0,0,0,...,-0.398685,-0.061430,-0.024931,-0.423288,1.592973,-0.883007,-0.101026,0.101860,1.279241,2
3,1,0,4,1,0,0.999745,1,0,1,0,...,0.178962,0.078587,-0.060135,-0.189770,0.138108,1.462135,0.084569,0.101860,-0.640618,3
4,1,0,4,0,0,0.256311,1,0,0,0,...,0.178962,-0.082432,-0.152204,-0.189770,0.138108,0.865190,-0.387855,0.506468,-1.241892,1


## 使用SVC进行训练

In [9]:
from time import time
import datetime
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score, recall_score

#### 划分训练集和测试集

In [10]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(jibing,jibing_res,test_size=0.3,random_state=90)

In [11]:
Xtrain.head()

,左右,是否外伤,症状持续时间,明显夜间痛,性别,年龄,高血压,高血脂,2型糖尿病,吸烟与否,...,腺苷脱氨酶ADA,果糖胺,肌酸激酶,α-L-盐藻糖苷酶,乳酸,淀粉酶,同型半胱氨酸,铁,总铁结合力,血型
1538,1,0,4,1,0,-0.412780,0,0,0,0,...,-0.109861,-0.024092,-0.130541,0.720952,0.389403,-0.328701,-0.522833,1.445158,0.530285,0
1161,1,0,3,0,1,-1.230557,0,0,1,0,...,0.467786,-0.024092,-0.062843,0.160508,0.138108,-1.010924,-0.505961,-0.496960,-0.714458,3
1566,1,0,3,1,0,0.107624,1,0,0,0,...,-0.976332,-0.035760,0.064430,0.627545,1.592973,0.140328,-0.084154,-0.901568,0.319312,3
785,1,0,3,1,0,0.330654,0,0,0,0,...,0.178962,0.094922,-0.122417,-0.189770,0.270368,-0.456617,-0.742173,1.040550,-0.334706,1
1087,1,1,1,0,0,-0.858840,1,0,0,0,...,0.467786,-0.033426,-0.106169,-0.773566,-0.919976,-0.499256,0.118314,0.409362,0.087241,0


In [12]:
Ytrain.head()

,结果
1538,0
1161,1
1566,0
785,0
1087,0


### 存在样本不均衡的问题
#### 但是不知道怎么处理

In [13]:
Ytrain.value_counts()

结果
0     983
1     153
dtype: int64

In [14]:
Ytest.value_counts()

结果
0     435
1      53
dtype: int64

### 训练集的统计信息
#### 没有过大或过小的数
#### 数据预处理完毕

In [15]:
Xtrain.describe()

,左右,是否外伤,症状持续时间,明显夜间痛,性别,年龄,高血压,高血脂,2型糖尿病,吸烟与否,...,腺苷脱氨酶ADA,果糖胺,肌酸激酶,α-L-盐藻糖苷酶,乳酸,淀粉酶,同型半胱氨酸,铁,总铁结合力,血型
count,1136.000000,1136.000000,1136.000000,1136.000000,1136.000000,1136.000000,1136.000000,1136.000000,1136.000000,1136.000000,...,1136.000000,1136.000000,1136.000000,1136.000000,1136.000000,1136.000000,1136.000000,1136.000000,1136.000000,1136.000000
mean,0.657570,0.213028,3.432218,0.484155,0.344190,-0.001208,0.345951,0.026408,0.117077,0.130282,...,-0.019350,0.010162,0.007161,0.025464,-0.001033,0.018567,-0.009966,0.008116,0.029660,1.573944
std,0.474731,0.409628,0.650842,0.499969,0.475313,1.112330,0.475887,0.160417,0.321654,0.336761,...,0.981620,1.194514,1.170069,1.079052,0.993298,1.011454,0.950631,1.021371,1.009216,1.230948
min,0.000000,0.000000,1.000000,0.000000,0.000000,-3.089142,0.000000,0.000000,0.000000,0.000000,...,-2.131627,-0.215448,-0.301411,-2.408194,-1.845799,-1.821064,-1.214597,-2.536185,-3.457114,0.000000
25%,0.000000,0.000000,3.000000,0.000000,0.000000,-0.487123,0.000000,0.000000,0.000000,0.000000,...,-0.687509,-0.075432,-0.125125,-0.656806,-0.655455,-0.669812,-0.522833,-0.658804,-0.651166,0.000000
50%,1.000000,0.000000,3.000000,0.000000,0.000000,0.033281,0.000000,0.000000,0.000000,0.000000,...,-0.109861,-0.038094,-0.075028,-0.073010,-0.126413,-0.158145,-0.168515,-0.092352,0.023949,2.000000
75%,1.000000,0.000000,4.000000,1.000000,1.000000,0.479341,1.000000,0.000000,0.000000,0.000000,...,0.467786,0.013246,0.004856,0.510785,0.402629,0.524078,0.236420,0.571205,0.667418,3.000000
max,1.000000,1.000000,4.000000,1.000000,1.000000,29.770634,1.000000,1.000000,1.000000,1.000000,...,6.244259,40.135092,30.875280,13.120778,14.951277,7.772699,10.494774,4.536363,4.253967,3.000000


### 训练

### 要调的参数
- 惩罚系数 C
- 核函数
- gamma

### 使用不同核函数

- 

In [59]:
clf = SVC(kernel='rbf', C=1e5,gamma='auto')
clf.fit(Xtrain, Ytrain)
y_pre = clf.predict(Xtest)
metrics_ = res_metrics(Ytest,y_pre)
metrics_

{'precision': 0.8866666666666667,
 'recall': 0.9236111111111112,
 'f1': 0.9047619047619048}